# Framework

## dependencies

In [51]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime

## load data

In [52]:
train_filepath = 'data/train.csv'
train_df = pd.read_table(train_filepath)

test_filepath = 'data/test.csv'
test_df = pd.read_table(test_filepath)

half_done_filepath = 'data/all_fucking_features.csv'
featured_df = pd.read_table(half_done_filepath)


/Users/zhangge/Documents/DB_proj/project-retepognajd/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/Users/zhangge/Documents/DB_proj/project-retepognajd/env/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """
/Users/zhangge/Documents/DB_proj/project-retepognajd/env/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


## functions for feature engineering

In [58]:
# to wash data
def wash_KC(x):
    if type(x) != str:
        x = 'Null'
    return x

#transfer string to date and time
def date_trans(str_time):
    if type(str_time) != str:
        return datetime.datetime(2000, 1, 1)
    format = "%Y-%m-%d %H:%M:%S.0"
    return datetime.datetime.strptime(str_time,format)

# create series of pandas date and time
def trans_date_time_series(data):
    cat = "Step Start Time"
    str_chosen_time = list(data[cat])
    date_time = [date_trans(x) for x in str_chosen_time]
    date = [x.date() for x in date_time]
    datetime_s = pd.to_datetime(date_time)
    date_s = pd.to_datetime(date)
    return (date_s,datetime_s)
  
# incorrects + corrects = # of steps
def count_num(row):
    return row["Corrects"]+row["Incorrects"]
    
# generate a dataframe for the relations of Sid, KC and Time
def gen_history_df(data):
    date,date_time = trans_date_time_series(data)

    data['Date and Time'] = date_time
    data['Date'] = date

    return data.groupby(['Anon Student Id','KC(Default)','Date']).size()
    
#     group_sum = grouped.aggregate(np.sum)
#     group_sum = group_sum[["Incorrects","Corrects"]]

#     temp_ = group_sum["Corrects"] + group_sum["Incorrects"]
#     group_sum["count"] = temp_
#     group_sum = group_sum[["count"]]
#     return group_sum

# applied to each step
def get_history_step(row):
    t = row["Row"]
    Sid = row["Anon Student Id"]
    KC = row["KC(Default)"]
    Date = row["Date"]
    day_thres = 7
    return historytable.loc[Sid].loc[KC].loc[Date]

## Wash Data, transfer NaN to other type

In [59]:
train_df['KC(Default)'] = train_df['KC(Default)'].apply(wash_KC)

test_df['KC(Default)'] = test_df['KC(Default)'].apply(wash_KC)

## Add Date, Date Time columns to data

In [60]:
train_df['Date and Time'],train_df['Date'] = trans_date_time_series(train_df)

## generate history table 

In [61]:
historytable = gen_history_df(train_df)
historytable.head()

Anon Student Id  KC(Default)           Date      
02i5jCrfQK       Changing axis bounds  2005-09-06     8
                                       2005-09-08     4
                                       2005-09-27     8
                                       2005-09-28    15
                                       2005-09-29     9
dtype: int64

In [62]:
train_df['KC History'] = train_df.apply(get_history_step,axis = 1)
train_df.head()

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default),Date and Time,Date,KC History
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,NaN,1,0,0,1,Null,NaN,2005-09-09 12:23:34,2005-09-09,19
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,...,NaN,1,0,0,1,Null,NaN,2005-09-09 12:24:07,2005-09-09,19
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,...,78.0,0,2,0,1,Define Variable,1,2005-09-09 12:24:22,2005-09-09,5
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,...,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1,2005-09-09 12:25:40,2005-09-09,5
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,...,NaN,1,0,0,1,Entering a given,1,2005-09-09 12:27:24,2005-09-09,20


## Functions for higher level feature

In [63]:
# the relation bewtween Correct answer and the number of hint
def student_ability_KC_hint(row):
    up = row["Corrects"]
    low = np.exp(row["Hints"])
    return up/low

# the relation bewtween Correct answer and the KC's frequency
def student_ability_KC_frequency(row):
    up = row["Corrects"]
    oppo = row["Opportunity(Default)"]
    if type(oppo) != str:
        oppo = 0
    else:
        oppo = oppo.split("~~")
        oppo = [int(x) for x in oppo]
        oppo = min(oppo)    
    return up / (oppo + int(row["KC History"] + 1))


def get_avg_KCF(row):
    Sid = row["Anon Student Id"]
    KC = row["KC(Default)"]
    try:
        return sum_KCH_KCF.loc[Sid].loc[KC]['AVG_KCF']
    except KeyError:
        return 0
        

def get_avg_KCH(row):
    Sid = row["Anon Student Id"]
    KC = row["KC(Default)"]
    try:
        return sum_KCH_KCF.loc[Sid].loc[KC]['AVG_KCH']
    except KeyError:
        return 0
    
def gen_ability_feature(data):
    data["KC_F"] = data.apply(student_ability_KC_frequency,axis = 1)
    data["KC_H"] = data.apply(student_ability_KC_hint,axis = 1)

    sum_KCH_KCF = data.groupby(["Anon Student Id","KC(Default)"])
    sum_KCH_KCF = sum_KCH_KCF.aggregate(np.sum)
    temp_count = sum_KCH_KCF["Corrects"] + sum_KCH_KCF["Incorrects"]
    sum_KCH_KCF["count"] = temp_count
    sum_KCH_KCF["AVG_KCF"] = sum_KCH_KCF["KC_F"] / sum_KCH_KCF["count"]
    sum_KCH_KCF["AVG_KCH"] = sum_KCH_KCF["KC_H"] / sum_KCH_KCF["count"]
    sum_KCH_KCF = sum_KCH_KCF[["AVG_KCH","AVG_KCF"]]
    
    return sum_KCH_KCF

## generate features representing the Students ability at KC

In [64]:
sum_KCH_KCF = gen_ability_feature(train_df)

train_df["ability from KC and Hints"] = train_df.apply(get_avg_KCH,axis = 1)
train_df["ability from KC and Frequency"] = train_df.apply(get_avg_KCF,axis = 1)

In [44]:
test_df["ability from KC and Hints"] = test_df.apply(get_avg_KCH,axis = 1)
test_df["ability from KC and Frequency"] = test_df.apply(get_avg_KCF,axis = 1)

## Save Featured Data

In [49]:
test_df.to_csv("data/merge_test.csv",sep='\t')
train_df.to_csv("data/merge_train.csv",sep='\t')

ZeroDivisionError: division by zero